In [1]:
import numpy as np
import pandas as pd

In [5]:
listofadj = []
listofpeerpressure = []
listofparticipation = []
for i in [1,2,3,4,6,9,10,12,15,19,20,21,23,24,25,28,29,31,32,33,36,37,39,41,42,43,45,46,47,48,50,51,52,55,57,59,60,62,64,65,66,67,68,70,71,72,73,75,77]:
    HHId = pd.read_csv('1. Network Data/Adjacency Matrix Keys/key_HH_vilno_' + str(i) + '.csv', header=None)
    adj = pd.read_csv('1. Network Data/Adjacency Matrices/adj_allVillageRelationships_HH_vilno_' + str(i) + '.csv', header=None)
    MF = pd.read_csv('3. Microfinance Participation/MF' + str(i) + '.csv', header=None)
    
    HHId[0] = HHId[0]+(i*1000)
    adj.columns=HHId[0].tolist()
    adj.index=HHId[0].tolist()
    MF.index=HHId[0].tolist()
    MF.index.name = 'hhid'
    MF.columns = ['Got Microfinance']
    
    subset = MF[MF['Got Microfinance'] == 1].index.tolist()
    subset = adj[subset]
    peerpressure = pd.DataFrame(subset.sum(axis=1), columns=['Friends Borrowed'])
    
    listofadj.append(adj)
    listofpeerpressure.append(peerpressure)
    listofparticipation.append(MF)
    
peerpressure = pd.concat(listofpeerpressure)
testy = pd.concat(listofparticipation)

In [6]:
Demographics = pd.read_csv('2. Demographics and Outcomes/household_characteristics.csv')
Demographics.set_index('hhid', inplace=True)

hohreligion = pd.get_dummies(Demographics['hohreligion'])
castesubcaste = pd.get_dummies(Demographics['castesubcaste'])
electricity = pd.get_dummies(Demographics['electricity'])
latrine = pd.get_dummies(Demographics['latrine'])
ownrent = pd.get_dummies(Demographics['ownrent'])

Demographics.drop(columns=['Unnamed: 0','village', 'adjmatrix_key', 'HHnum_in_village', 'rooftypeoth', 'hohreligion', 'castesubcaste', 'electricity', 'latrine', 'ownrent'], inplace=True)
Demographics = Demographics.join(hohreligion).join(castesubcaste).join(electricity).join(latrine).join(ownrent)

In [8]:
testx = pd.merge(peerpressure, Demographics, left_index=True, right_index=True, how='inner')
testx

,Friends Borrowed,rooftype1,rooftype2,rooftype3,rooftype4,rooftype5,room_no,bed_no,hhSurveyed,leader,...,Common,None,Owned,0.0,6.0,GIVEN BY GOVERNMENT,LEASED,OWNED,OWNED BUT SHARED,RENTED
1001,3,0,1,0,0,0,3,4,0,0,...,0,1,0,0,0,0,0,1,0,0
1002,0,0,1,0,0,0,1,1,1,1,...,0,1,0,0,0,0,0,1,0,0
1003,0,0,0,0,0,1,3,4,0,1,...,0,1,0,0,0,0,0,1,0,0
1004,1,0,1,0,0,0,2,6,0,0,...,0,0,1,0,0,0,0,1,0,0
1005,4,0,1,0,0,0,3,4,0,0,...,0,1,0,0,0,0,0,1,0,0
1006,4,0,0,1,0,0,2,1,1,0,...,0,0,1,0,0,0,0,1,0,0
1007,4,0,0,1,0,0,3,5,1,0,...,0,0,1,0,0,0,0,1,0,0
1008,4,0,0,0,1,0,2,1,1,0,...,0,0,1,0,0,0,0,1,0,0
1009,1,0,1,0,0,0,2,7,0,0,...,0,1,0,0,0,1,0,0,0,0
1010,0,0,0,1,0,0,2,1,0,0,...,0,1,0,0,0,0,0,1,0,0


In [9]:
from sklearn import linear_model as bestfit
model = bestfit.Lasso(alpha=0.1)
model.fit(testx,testy)
print(model.coef_)

[ 0.02170964  0.          0.         -0.          0.         -0.
 -0.         -0.         -0.          0.          0.         -0.
  0.         -0.          0.         -0.          0.          0.
  0.          0.         -0.         -0.          0.         -0.
  0.         -0.          0.          0.         -0.          0.
  0.        ]


In [32]:
listofvillagex = []
listofvillagey = []
for i in [1,2,3,4,6,9,10,12,15,19,20,21,23,24,25,28,29,31,32,33,36,37,39,41,42,43,45,46,47,48,50,51,52,55,57,59,60,62,64,65,66,67,68,70,71,72,73,75,77]:
    villagex = testx.loc[((i*1000) < testx.index) & (testx.index < (((i+1)*1000)-1))]
    villagey = testy.loc[((i*1000) < testy.index) & (testy.index < (((i+1)*1000)-1))]
    meanvillagex = pd.DataFrame(np.mean(villagex, axis=0),columns = [i])
    meanvillagey = pd.DataFrame(np.mean(villagey, axis=0),columns = [i])
    listofvillagex.append(meanvillagex)
    listofvillagey.append(meanvillagey)
testx2 = pd.concat(listofvillagex, axis=1)
testy2 = pd.concat(listofvillagey, axis=1)

In [46]:
testy2 = np.log(testy2 / (1-testy2))
testy2

,1,2,3,4,6,9,10,12,15,19,...,65,66,67,68,70,71,72,73,75,77
Got Microfinance,-1.203973,-1.704748,-1.899748,-2.569464,-1.269761,-1.524881,-2.154665,-0.806806,-2.021548,-1.349927,...,-1.597365,-1.93968,-0.869604,-1.784155,-1.356081,-1.554088,-1.715386,-1.568616,-1.193922,-2.233592


In [47]:
model2 = bestfit.Lasso(alpha=0.1)
model2.fit(testx2.T,testy2.T)
print(model2.coef_)

[ 0.47468105  0.         -0.          0.         -0.         -0.
 -0.         -0.         -0.         -0.          0.         -0.
  0.         -0.          0.         -0.         -0.         -0.
  0.          0.         -0.         -0.         -0.          0.
  0.          0.         -0.         -0.          0.          0.
  0.        ]


In [54]:
#n - number of images in dataset
#h - height of images in dataset
#w - width of images in dataset
#q - number of color bins

def calculate_w(binned_images): #binned_images is of dimensions n.h.w.q
    gaussiankernel = 5
    lamda = 0.5
    Q = binned_images.shape[3]
    p = binned_images.sum(axis=0).sum(axis=0).sum(axis=0)
    p = p / p.sum()
    #https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve2d.html
    return p

n, h, w, q = 100, 30, 30, 133
dummydata = np.random.rand(n,h,w,q)
w = calculate_w(dummydata)
w

array([0.0075089 , 0.00753491, 0.00750994, 0.00750435, 0.00753742,
       0.00750134, 0.00750393, 0.0075265 , 0.00752445, 0.00752339,
       0.00751956, 0.00750424, 0.00753167, 0.00754333, 0.00753602,
       0.00754349, 0.00752383, 0.00751865, 0.00751292, 0.00751511,
       0.00750163, 0.00753155, 0.00751242, 0.00750998, 0.00752716,
       0.00751267, 0.00752413, 0.00749743, 0.00752704, 0.00751412,
       0.00750658, 0.00751233, 0.00753342, 0.00750633, 0.00753027,
       0.00751667, 0.00753412, 0.00750748, 0.00751127, 0.00752144,
       0.00752644, 0.00750923, 0.0075052 , 0.00749815, 0.00749882,
       0.00750942, 0.00752915, 0.00752618, 0.00753443, 0.00751041,
       0.00752662, 0.00751351, 0.00751555, 0.00751741, 0.0075227 ,
       0.00751231, 0.00753246, 0.00754975, 0.00753172, 0.00751432,
       0.00749717, 0.00751615, 0.0075207 , 0.0075332 , 0.00750181,
       0.00752719, 0.00753666, 0.00752223, 0.00751331, 0.0075208 ,
       0.00752638, 0.00751985, 0.00753078, 0.00750287, 0.00751